# 단일 모델을 이용하여 회귀모델분석하기 2
## 집값과 범죄율 사이의 관계를 단일회귀모델을 이용하여 세우기
### 16.03.29 실습 수행, by 권도형(DOHK)

## 데이터셋 불러오기

In [2]:
import graphlab

In [3]:
sales = graphlab.SFrame('kc_house_data.gl/')

# graphlab을 쓰지 않는다고 할 때, dataset 전체에 해당하는 파일 1개, 
# test set을 위한 파일 1개, train set을 위한 파일 1개 이렇게 세 개가 각각 필요하게 된다.

# ----------------- #
# SFrame자료구조와 같은 자료구조를 내가 직접 만들어야 할까?
# ----------------- #

2016-04-04 15:59:13,594 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: C:\Users\dato\AppData\Local\Temp\graphlab_server_1459753152.log.0


This non-commercial license of GraphLab Create is assigned to dohk@koreatech.ac.kr and will expire on January 17, 2017. For commercial licensing options, visit https://dato.com/buy/.


In [4]:
sales

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650,1,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242,2,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000,1,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000,1,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080,1,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930,1,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819,2,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711,1,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470,1,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560,2,0


In [5]:
train_set, test_set = sales.random_split(.8, seed=0)

# ----------------- #
# seed가 0이라는 것은 무슨 의미일까? 
# seed는 한마디로 초기값을 의미한다. seed(씨앗, 초기값)에서부터 난수가 생성되기 시작한다.
# 실제 구현시엔 train_set과 test_set 나누기를 어떻게 해야할까?
# ----------------- #

In [6]:
prices = sales['price']

# 가격에 대한 정보만을 따로 저장하게 한다. sales는 SFrame, 즉 table형태의 자료구조이다.
# 반면 prices는 SFrame으로부터 떨어져나온 SArray다.

# ----------------- #
# 실제 구현시엔 어떻게 특정 자료구조의 내용만을 따로 저장하는 것을 가능하게 할까?
# ----------------- #

##기본적인 내용에 대하여
### SArray를 이용한 sum, mean 연산

In [9]:
sum_prices = prices.sum()
sum_prices

11672925011.0

In [11]:
num_houses = prices.size()
num_houses

21613

In [12]:
avg_prices_method1 = sum_prices/num_houses
avg_prices_method1

540088.1419053348

In [13]:
avg_prices_method2 = prices.mean()
avg_prices_method2

540088.1419053345

### SArray를 이용한 scala-mutiply 연산

In [15]:
prices

dtype: float
Rows: 21613
[221900.0, 538000.0, 180000.0, 604000.0, 510000.0, 1225000.0, 257500.0, 291850.0, 229500.0, 323000.0, 662500.0, 468000.0, 310000.0, 400000.0, 530000.0, 650000.0, 395000.0, 485000.0, 189000.0, 230000.0, 385000.0, 2000000.0, 285000.0, 252700.0, 329000.0, 233000.0, 937000.0, 667000.0, 438000.0, 719000.0, 580500.0, 280000.0, 687500.0, 535000.0, 322500.0, 696000.0, 550000.0, 640000.0, 240000.0, 605000.0, 625000.0, 775000.0, 861990.0, 685000.0, 309000.0, 488000.0, 210490.0, 785000.0, 450000.0, 1350000.0, 228000.0, 345000.0, 600000.0, 585000.0, 920000.0, 885000.0, 292500.0, 301000.0, 951000.0, 430000.0, 650000.0, 289000.0, 505000.0, 549000.0, 425000.0, 317625.0, 975000.0, 287000.0, 204000.0, 1325000.0, 1040000.0, 325000.0, 571000.0, 360000.0, 349000.0, 832500.0, 380000.0, 480000.0, 410000.0, 720000.0, 390000.0, 360000.0, 355000.0, 356000.0, 315000.0, 940000.0, 305000.0, 461000.0, 215000.0, 335000.0, 243500.0, 1099880.0, 153000.0, 430000.0, 700000.0, 905000.0, 247500.0

In [14]:
multiplying_with_constant = 0.5*prices
multiplying_with_constant

dtype: float
Rows: 21613
[110950.0, 269000.0, 90000.0, 302000.0, 255000.0, 612500.0, 128750.0, 145925.0, 114750.0, 161500.0, 331250.0, 234000.0, 155000.0, 200000.0, 265000.0, 325000.0, 197500.0, 242500.0, 94500.0, 115000.0, 192500.0, 1000000.0, 142500.0, 126350.0, 164500.0, 116500.0, 468500.0, 333500.0, 219000.0, 359500.0, 290250.0, 140000.0, 343750.0, 267500.0, 161250.0, 348000.0, 275000.0, 320000.0, 120000.0, 302500.0, 312500.0, 387500.0, 430995.0, 342500.0, 154500.0, 244000.0, 105245.0, 392500.0, 225000.0, 675000.0, 114000.0, 172500.0, 300000.0, 292500.0, 460000.0, 442500.0, 146250.0, 150500.0, 475500.0, 215000.0, 325000.0, 144500.0, 252500.0, 274500.0, 212500.0, 158812.5, 487500.0, 143500.0, 102000.0, 662500.0, 520000.0, 162500.0, 285500.0, 180000.0, 174500.0, 416250.0, 190000.0, 240000.0, 205000.0, 360000.0, 195000.0, 180000.0, 177500.0, 178000.0, 157500.0, 470000.0, 152500.0, 230500.0, 107500.0, 167500.0, 121750.0, 549940.0, 76500.0, 215000.0, 350000.0, 452500.0, 123750.0, 99500.

### SArray를 이용한 SArray-multiply 연산

In [17]:
prices_squared = prices*prices
prices_squared

dtype: float
Rows: 21613
[49239610000.0, 289444000000.0, 32400000000.0, 364816000000.0, 260100000000.0, 1500625000000.0, 66306250000.0, 85176422500.0, 52670250000.0, 104329000000.0, 438906250000.0, 219024000000.0, 96100000000.0, 160000000000.0, 280900000000.0, 422500000000.0, 156025000000.0, 235225000000.0, 35721000000.0, 52900000000.0, 148225000000.0, 4000000000000.0, 81225000000.0, 63857290000.0, 108241000000.0, 54289000000.0, 877969000000.0, 444889000000.0, 191844000000.0, 516961000000.0, 336980250000.0, 78400000000.0, 472656250000.0, 286225000000.0, 104006250000.0, 484416000000.0, 302500000000.0, 409600000000.0, 57600000000.0, 366025000000.0, 390625000000.0, 600625000000.0, 743026760100.0, 469225000000.0, 95481000000.0, 238144000000.0, 44306040100.0, 616225000000.0, 202500000000.0, 1822500000000.0, 51984000000.0, 119025000000.0, 360000000000.0, 342225000000.0, 846400000000.0, 783225000000.0, 85556250000.0, 90601000000.0, 904401000000.0, 184900000000.0, 422500000000.0, 83521000000.0

### SArray를 이용한 SS(sum of squares:제곱합) 연산

In [19]:
sum_prices_squares = prices_squared.sum()
sum_prices_squares

9217325133550736.0

## gradient가 적용된 simple linear regression 모델 만들기

In [130]:
def simple_linear_regression(input_feature, output):
# xi의 시그마, yi의 시그마 구하기
    N = input_feature.size()
    sum_output = output.sum()                                         # sum of Y
    sum_input_feature = input_feature.sum()                           # sum of X
    product_input_output = input_feature*output
    squared_input_feature = input_feature*input_feature
    sum_product_input_output = product_input_output.sum()             # sum of X*Y
    sum_squared_input_feature = squared_input_feature.sum()           # sum of X^2
    squared_sum_input_feature = sum_input_feature*sum_input_feature   # (sum of X) * (sum of X)

    print "length of feature : " + str(N)
    
    numerator = (sum_product_input_output)-(1/N)*((sum_input_feature)*(sum_output))
    denominator = (sum_squared_input_feature) - (1/N)*((sum_input_feature)*(sum_input_feature))
    slope = numerator/denominator
    
    mean_sum_output = sum_output/N
    mean_sum_input_feature = sum_input_feature/N
    
    intercept = mean_sum_output - ((slope)*(mean_sum_input_feature))
    
    
#    intercept = ((1/N)*(sum_output)-((sum_product_input_output
#                       -((1/N)*sum_product_input_output)/((sum_squared_input_feature
#                                    -(1/N)*squared_sum_input_feature))))*(1/N)*(sum_input_feature))
             
#    print "test - " + str(sum_input_feature)
#    print "test - " + str(sum_output)
#    print "test - " + str(squared_sum_input_feature)
#    print "test - " + str(product_input_output)
#    print "test - " + str(sum_product_input_output)
#    print "test - " + str(squared_input_feature)
#    print "test - " + str(sum_squared_input_feature)

    return (intercept, slope)

In [131]:
# 간단한 모수 추정. 성공했는지 확인
test_feature = graphlab.SArray(range(5))
test_output = graphlab.SArray(1 + 1*test_feature)
(test_intercept,test_slope) = simple_linear_regression(test_feature, test_output)
print "Intercept : " + str(test_intercept)
print "Slope : " + str(test_slope)

length of feature : 5
Intercept : 1
Slope : 1


In [132]:
# 80%의 훈련집합으로 모델을 훈련시켜 모수를 구함
sqft_intercept, sqft_slope = simple_linear_regression(train_set['sqft_living'], train_set['price'])

print "Intercept :" + str(sqft_intercept)
print "Slope : " + str(sqft_slope)

length of feature : 17384
Intercept :-7731.6858275
Slope : 263.024303956


In [133]:
def get_regression_predictions(input_feature, intercept, slope):
    product_slope_feature = slope*(input_feature)    
    predicted_values = intercept + product_slope_feature # 예측치 구하기 y=w0+w1*x
    
    return predicted_values

In [140]:
my_house_sqft = 2650
estimated_price = get_regression_predictions(my_house_sqft,sqft_intercept, sqft_slope)
print "%d평방피트인 집값의 예측치는 $%.2f" % (my_house_sqft, estimated_price)

2650평방피트인 집값의 예측치는 $689282.72


## 만든 모델에 대해 RSS를 이용하여 평가하기

In [141]:
def get_residual_sum_of_squares(input_feature, output, intercept, slope):
    product_slope_feature = slope*(input_feature)    
    predicted_values = intercept + product_slope_feature # 예측치
    
    residuals = output-predicted_values # "잔차"구하기;빼는 순서는 상관없다. 어자피 square되기 때문에.
    
    squared_residuals = residuals*residuals # "잔차제곱"구하기
    sum_squared_residuals = squared_residuals.sum() # "잔차제곱합(RSS)"구하기
    
    RSS = sum_squared_residuals
    
    return(RSS)

In [142]:
print get_residual_sum_of_squares(test_feature,test_output,test_intercept,test_slope)
# 정확히 회귀선 위에 놓인 데이터로 RSS를 구하는 중이기 때문에 0이 나와야 정상이다.

0


In [144]:
rss_prices_on_sqft = get_residual_sum_of_squares(train_set['sqft_living'],
                                                train_set['price'],
                                                sqft_intercept,
                                                sqft_slope)
print '면적에 대한 가격을 예측해본 결과에 대한 RSS는 ' + str(rss_prices_on_sqft)

면적에 대한 가격을 예측해본 결과에 대한 RSS는 1.2072119188e+15


## 가격이 주어졌을 때 feature를 예측하기

In [148]:
# 함수관계; y = a + b*x가 주어진 상태이므로 x도 예측 가능하다.
# 기울기:b, 절편:a, 가격:y
# x = ?

def inverse_regression_predictions(output, intercept, slope):
    estimated_feature = (output-intercept)*(1/slope)
    
    return estimated_feature

In [151]:
my_house_price = 800000
estimated_squarefeet = inverse_regression_predictions(my_house_price, sqft_intercept, sqft_slope)
print "가격이 %.2f인 집의 평방피트는 %d피트로 예상됨" % (my_house_price, estimated_squarefeet)

가격이 800000.00인 집의 평방피트는 3070피트로 예상됨
